# Camada raw
## Armazenando dados originais

Importando biblotecas necessárias

In [2]:
import pandas as pd

Lendo arquivo original em csv

In [18]:
rwzd_olist_order_reviews = pd.read_csv(
    filepath_or_buffer='../dataset/incoming/olist_order_reviews_dataset.csv',
    delimiter=',',
    quotechar='"',
    header=0,
    dtype={
        'review_id': str,
        'order_id': str,
        'review_score': pd.UInt8Dtype(),
        'review_comment_title': str,
        'review_comment_message': str,
        'review_creation_date': str,
        'review_answer_timestamp': str,
    },
    date_format='%Y-%m-%d %H:%M:%s',
    parse_dates=['review_creation_date', 'review_answer_timestamp'],
)

Assegurando tipo _datetime_ para colunas com datas

In [19]:
rwzd_olist_order_reviews['review_creation_date'] = rwzd_olist_order_reviews['review_creation_date'].astype('datetime64[ns]')
rwzd_olist_order_reviews['review_answer_timestamp'] = rwzd_olist_order_reviews['review_answer_timestamp'].astype('datetime64[ns]')


Visualizando informações sobre os dados

In [20]:
rwzd_olist_order_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 7 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   review_id                100000 non-null  object        
 1   order_id                 100000 non-null  object        
 2   review_score             100000 non-null  UInt8         
 3   review_comment_title     11715 non-null   object        
 4   review_comment_message   41753 non-null   object        
 5   review_creation_date     100000 non-null  datetime64[ns]
 6   review_answer_timestamp  100000 non-null  datetime64[ns]
dtypes: UInt8(1), datetime64[ns](2), object(4)
memory usage: 4.8+ MB


In [22]:
rwzd_olist_order_reviews.tail()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
99995,f3897127253a9592a73be9bdfdf4ed7a,22ec9f0669f784db00fa86d035cf8602,5,NaN,NaN,2017-12-09,2017-12-11 20:06:42
99996,b3de70c89b1510c4cd3d0649fd302472,55d4004744368f5571d1f590031933e4,5,NaN,"Excelente mochila, entrega super rápida. Super...",2018-03-22,2018-03-23 09:10:43
99997,1adeb9d84d72fe4e337617733eb85149,7725825d039fc1f0ceb7635e3f7d9206,4,NaN,NaN,2018-07-01,2018-07-02 12:59:13
99998,be360f18f5df1e0541061c87021e6d93,f8bd3f2000c28c5342fedeb5e50f2e75,1,NaN,Solicitei a compra de uma capa de retrovisor c...,2017-12-15,2017-12-16 01:29:43
99999,efe49f1d6f951dd88b51e6ccd4cc548f,90531360ecb1eec2a1fbb265a0db0508,1,NaN,"meu produto chegou e ja tenho que devolver, po...",2017-07-03,2017-07-03 21:01:49


Salvando dados na camada raw em formato parquet

In [23]:
rwzd_olist_order_reviews.to_parquet(
    path='../dataset/raw/rwzd_olist_order_reviews.parquet.snappy',
    engine='pyarrow',
    compression='snappy',
    index=False,
)

# Camada trusted
## Aplicando tratamentos nos dados

Importando bibliotecas necessárias

In [70]:
import pandas as pd

In [71]:
pd.set_option('display.max_colwidth', None)

Lendo dados da camada raw

In [72]:
trzd_olist_order_reviews = pd.read_parquet(
    path='../dataset/raw/rwzd_olist_order_reviews.parquet.snappy',
    engine='pyarrow',
)

Filtrando por avaliações que tenham ao menos uma mensagem, mas não necessariamente o título

In [73]:
trzd_olist_order_reviews.dropna(thresh=6, inplace=True)

Filtrando avaliações que comentaram sobre entregas nos comentários

In [74]:
deliver_message_mask = trzd_olist_order_reviews['review_comment_message'].str.contains(
    pat='entrega|prazo|recebi|recebido|chego|chegou',
    case=False,
    regex=True,
    na=False,
)

Filtrando avaliações que comentaram sobre entregas no título

In [75]:
deliver_title_mask = trzd_olist_order_reviews['review_comment_title'].str.contains(
    pat='entrega|prazo|recebi|recebido|chego|chegou',
    case=False,
    regex=True,
    na=False,
)

Criando máscara para identificar avaliações que comentaram sobre entregas tanto no título quanto nos comentários

In [76]:
deliver_mask = (deliver_message_mask | deliver_title_mask)

Removendo avaliações sobre entregas uma vez que o foco está nas avaliações de produtos

In [77]:
deliver_reviews_index = trzd_olist_order_reviews.loc[deliver_mask].index

In [78]:
trzd_olist_order_reviews.drop(
    index=deliver_reviews_index,
    inplace=True
)

In [81]:
trzd_olist_order_reviews.tail()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
99967,0e7bc73fde6782891898ea71443f9904,bd78f91afbb1ecbc6124974c5e813043,4,👍,Aprovado!,2018-07-04,2018-07-05 00:25:13
99971,58be140ccdc12e8908ff7fd2ba5c7cb0,0ebf8e35b9807ee2d717922d5663ccdb,5,muito bom produto,"Ficamos muito satisfeitos com o produto, atende às nossas necessidades com muita facilidade. Recomendo!",2018-06-30,2018-07-02 23:09:35
99974,40743b46a0ee86375cedb95e82b78d75,3e93213bb8fdda91186b4018b2fe0030,5,OTIMA EMBALAGEM,None,2018-08-08,2018-08-08 16:56:16
99975,2ee221b28e5b6fceffac59487ed39348,f2d12dd37eaef72ed7b1186b2edefbcd,2,Foto enganosa,Foto muito diferente principalmente a graninha sintética muito rala...diferente da foto,2018-03-28,2018-05-25 01:23:26
99990,a709d176f59bc3af77f4149c96bae357,d5cb12269711bd1eaf7eed8fd32a7c95,3,None,"O produto não foi enviado com NF, não existe venda sem NF, com certeza fico no aguardo do envio da NF podendo ser por e-mail.",2018-05-19,2018-05-20 21:51:06


Salvando dados na camada trusted em formato parquet

In [82]:
trzd_olist_order_reviews.to_parquet(
    path='../dataset/trusted/trzd_olist_order_reviews.parquet.snappy',
    engine='pyarrow',
    compression='snappy',
    index=False,
)